<a href="https://colab.research.google.com/github/hwankang/Keyword-detection/blob/master/jan26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
!tar -zxvf speech_commands_v0.02.tar.gz


In [ ]:

!pip install python_speech_features
from os import listdir
from os.path import isdir, join
import librosa
import random
import numpy as np
import matplotlib.pyplot as plt
import python_speech_features


In [ ]:
dataset_path='/content'
for name in listdir(dataset_path):
    if isdir(join(dataset_path, name)):
        print(name)


In [ ]:
all_targets=[name for name in listdir(dataset_path)  if isdir(join(dataset_path,name))]
print(all_targets)


In [ ]:
all_targets.remove('_background_noise_')
all_targets.remove('.config')
print(all_targets)

In [ ]:
all_targets.remove('sample_data')
print(all_targets)

In [ ]:
# see how many files are in each 
num_samples = 0
for target in all_targets:
    print(len(listdir(join(dataset_path,target))))
    #폴더의 내용 
    num_samples += len(listdir(join(dataset_path, target)))
print('Total samples=', num_samples)

In [ ]:
target_list = all_targets
feature_sets_file = 'all_targets_mfcc_sets_0123.npz'
#perc_keep_samples =0.1
#val_ratio = 0.1
#test_ratio = 0.1
perc_keep_samples =0.1
val_ratio = 0.1
test_ratio = 0.1
#sample_rate = 8000
sample_rate = 16000
#num_mfcc = 40
num_mfcc = 20
#num_mfcc = 16
#len_mfcc = 16
len_mfcc = 49

In [ ]:
filenames=[]
y=[]
for index, target in enumerate(target_list):
    print(join(dataset_path,target))
    filenames.append(listdir(join(dataset_path, target)))
    y.append(np.ones(len(filenames[index]))*index)

In [ ]:
#정답벡터 y 검사 
#정답벡터 y 검사 
print(y)
for item in y:
    print(len(item))

In [ ]:
filenames = [item for sublist in filenames for item in sublist]
y = [item for sublist in y for item in sublist]

In [ ]:
filenames_y=list(zip(filenames,y))
random.shuffle(filenames_y)
filenames,y=zip(*filenames_y)


In [ ]:
print(len(filenames))
filenames=filenames[:int(len(filenames)*perc_keep_samples)]
print(len(filenames))

In [ ]:
val_set_size =int(len(filenames)*val_ratio)
test_set_size =int(len(filenames)*test_ratio)

In [ ]:

filenames_val=filenames[:val_set_size]
filenames_test=filenames[val_set_size:(val_set_size + test_set_size)]
filenames_train=filenames[(val_set_size + test_set_size):]

In [ ]:
#y를 훈련, 유효 및 테스트 집합으로 분리 
y_orig_val=y[:val_set_size]
y_orig_test=y[val_set_size:(val_set_size + test_set_size)]
y_orig_train=y[(val_set_size + test_set_size):]

In [ ]:
# 함수 MFCC 만들기 
def calc_mfcc(path):
    signal, fs = librosa.load(path,sr=sample_rate)
    mfccs = python_speech_features.base.mfcc(signal,
                                           samplerate=fs,
                                           #winlen=0.256,
                                           winlen=0.040,
                                           #winstep=0.050,
                                           winstep=0.020,
                                           numcep=num_mfcc,
                                           #nfilt=26,
                                           #nfilt=40,
                                           nfilt=20,
                                           nfft=2048,
                                           preemph=0.0,
                                           ceplifter=0,
                                           appendEnergy=False,
                                           winfunc=np.hanning)
    return mfccs.transpose()

In [ ]:
prob_cnt = 0
x_test = []
y_test = []
for index, filename in enumerate(filenames_train):
    #if index>=500:
    #if index>=100:
    #    break
    path = join(dataset_path, target_list[int(y_orig_train[index])],
               filename)
    
    mfccs =calc_mfcc(path)
    
    if mfccs.shape[1]==len_mfcc:
        x_test.append(mfccs)
        y_test.append(y_orig_train[index])
    else:
        print('Dropped:',index, mfccs.shape)
        prob_cnt +=1

In [ ]:
print('% of problematic samples', prob_cnt/500)

In [ ]:
!pip install playsound
from playsound import playsound
idx=10
path=join(dataset_path, target_list[int(y_orig_train[idx])],
         filenames_train[idx])
mfccs = calc_mfcc(path)
print("MFCCs:",mfccs)
fig = plt.figure()
plt.imshow(mfccs, cmap='inferno',origin='lower')
print(target_list[int(y_orig_train[idx])])
playsound(path)

In [ ]:
def extract_features(in_files, in_y):
    prob_cnt = 0
    out_x = []
    out_y = []
    for index, filename in enumerate(in_files):
        path = join(dataset_path,target_list[int(in_y[index])],
                    filename)
        if not path.endswith('.wav'):
            continue
            
    
        mfccs =calc_mfcc(path)
    
        if mfccs.shape[1]==len_mfcc:
            out_x.append(mfccs)
            out_y.append(in_y[index])
        else:
            print('Dropped:',index, mfccs.shape)
            prob_cnt +=1
    return out_x, out_y, prob_cnt

In [ ]:
x_train, y_train, prob = extract_features(filenames_train,
                                         y_orig_train)
print( 'Removed percentage:', prob/len(y_orig_train))
x_val, y_val, prob = extract_features(filenames_val,
                                         y_orig_val)
print( 'Removed percentage:', prob/len(y_orig_val))
x_test, y_test, prob = extract_features(filenames_test,
                                         y_orig_test)
print( 'Removed percentage:', prob/len(y_orig_test))

In [ ]:
np.savez(feature_sets_file,
       x_train=x_train,
       y_train=y_train,
        x_val=x_val,
        y_val=y_val,
        x_test=x_test,
        y_test=y_test)

In [ ]:
feature_sets = np.load(feature_sets_file)
feature_sets.files

In [ ]:
len(feature_sets['x_train'])
len(feature_sets['y_train'])

In [ ]:
print(feature_sets['y_val'])

In [ ]:
len(feature_sets['x_val'])

In [ ]:
len(feature_sets['x_test'])

In [ ]:
!wget https://github.com/ARM-software/ML-KWS-for-MCU/archive/master.zip
!unzip master.zip

In [ ]:
feature_sets_path='/content'
feature_sets_filename = 'all_targets_mfcc_sets_0123.npz'

In [ ]:
feature_sets=np.load(join(feature_sets_path, feature_sets_filename))
print(feature_sets.files)

In [ ]:
x_train=feature_sets['x_train']
y_train=feature_sets['y_train']
x_val=feature_sets['x_val']
y_val=feature_sets['y_val']
x_test=feature_sets['x_test']
y_test=feature_sets['y_test']

In [ ]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
print(y_val)

In [ ]:
x_train=x_train.astype('float64')
y_train=y_train.astype('float64')
x_val=x_val.astype('float64')
y_val=y_val.astype('float64')
x_test=x_test.astype('float64')
y_test=y_test.astype('float64')

In [ ]:
import tensorflow as tf
model = tf.keras.models.Sequential([
  #tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Flatten(input_shape=(20,49)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(38, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=100)

In [ ]:
model.evaluate(x_test, y_test)


In [ ]:
x_train = x_train.reshape(x_train.shape[0],
                         x_train.shape[1],
                         x_train.shape[2],
                         1)
x_val = x_val.reshape(x_val.shape[0],
                         x_val.shape[1],
                         x_val.shape[2],
                         1)
x_test = x_test.reshape(x_test.shape[0],
                         x_test.shape[1],
                         x_test.shape[2],
                         1)
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

In [ ]:
#https://www.geeksforgeeks.org/python-image-classifiction-using keras/
from tensorflow.keras import layers, models
sample_shape=(20,49,1)
model = models.Sequential()
model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=sample_shape, padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, (2, 2), activation='relu',padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, (2, 2), activation='relu',padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
#model.add(layers.Dense(1, activation='softmax'))
model.add(layers.Dense(38, activation='sigmoid'))
model.summary()

In [ ]:
sample_shape=x_test.shape[1:]
print(sample_shape)

In [ ]:
#https://www.geeksforgeeks.org/python-image-classifiction-using keras/
#skip
model = models.Sequential()
model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=sample_shape, padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(32, (2, 2), activation='relu',padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(64, (2, 2), activation='relu',padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
#model.add(layers.Dense(1, activation='softmax'))
model.add(layers.Dense(38, activation='sigmoid'))
model.summary()

In [ ]:
#model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam')
model.compile(optimizer='adam', loss=['sparse_categorical_crossentropy'], metrics=['sparse_categorical_accuracy'])
#model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',learning_rate=0.01,metrics=['acc'])

In [ ]:
history=model.fit(x_train,y_train,epochs=30,batch_size=100,
                 validation_data=(x_val,y_val))

In [ ]:
import matplotlib.pyplot as plt
#epochs=5
acc=history.history['sparse_categorical_accuracy']
val_acc=history.history['val_sparse_categorical_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(acc)+1)
plt.plot(epochs,acc,'bo', label='Training acc')
plt.plot(epochs,val_acc,'b', label='Validation acc')
plt.title('training and valodation accuracy')
plt.legend()

plt.figure()


plt.plot(epochs,loss,'bo', label='Training loss')
plt.plot(epochs,val_loss,'b', label='Validation loss')
plt.title('training and valodation loss')
plt.legend()

plt.show()


In [ ]:
import math
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.4
    epochs_drop = 15.0
    lrate = initial_lrate * math.pow(drop,  
            math.floor((1+epoch)/epochs_drop))
    
    if (lrate < 4e-5):
        lrate = 4e-5
      
    print('Changing learning rate to {}'.format(lrate))
    return lrate
lrate = LearningRateScheduler(step_decay)

In [ ]:
earlystopper = EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=10,
                             verbose=1, restore_best_weights=True)
checkpointer = ModelCheckpoint('model-attRNN.h5', monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True)

results = model.fit(trainGen, validation_data=valGen, epochs=60, use_multiprocessing=False, workers=4, verbose=2,
                    callbacks=[earlystopper, checkpointer, lrate])

model.save('model-attRNN.h5')

In [ ]:
# summarize history for categorical accuracy
plt.plot(results.history['sparse_categorical_accuracy'])
plt.plot(results.history['val_sparse_categorical_accuracy'])
plt.title('Categorical accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,len(acc)+1)
plt.plot(epochs,acc,'bo', label='Training acc')
plt.plot(epochs,val_acc,'b', label='Validation acc')
plt.title('training and valodation accuracy')
plt.legend()

plt.figure()



plt.plot(epochs,loss,'bo', label='Training loss')
plt.plot(epochs,val_loss,'b', label='Validation loss')
plt.title('training and valodation loss')
plt.legend()

plt.show()